<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Dynamic systems: 1. Introduction 
# Introduction to openFoam
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

`openFOAM` is an open-source software package for solving partial differential equations.

The name openFOAM means 
${\bf open}\mbox{ }Source\mbox{ }{\bf F}ield\mbox{ }{\bf O}peration\mbox{ }{\bf A}nd\mbox{ }{\bf M}anipulation$.

The package is a compilating of solvers for partial differential equations, mainly centered around the
**Navier-Stokes equation** and associated transport problems, thus applicable to a wide range of
problems from **computational fluid dynamics** (CFD), e.g. flow in liquid and gas phases, multi-phase flow,
as well as elastic deformation problems.

Versions:
- [openFoam foundation](https://openfoam.org)
- [Company](https://www.openfoam.com)

Use openfoam9, if you want to be compatible with this lecture!

----
## Typical problem

A typical problem solved with openFoam is the **lid-driven cavity** problem from fluid dynamics.

<img src="images/cavity.jpg" style="width:10cm">

A box with dimensions $l_x$, $l_y$, and $l_z$ [m] is filled with a fluid of density $\rho$ [kg/m$^3$] and
kinematic viscosity $\nu$ [m$^2$/s].

The top surface to moved to the right with a constant velocity $u_{top}$ [m/s].

We want to calculate both 
- the **velocity** $u(x,y,z)$ [m/s]
- the **pressure** $p(x,y,z)$ [Pa]

in the model domain.

We have to solve both the continuity equation and the equation of motion. We define the fluid as **incompressible**.
We then have as differential equations:

**Continuity equation**
$$
\nabla \cdot u = 0
$$
**Equation of motion**
$$
\frac{\partial u}{\partial t}
+ u \cdot \nabla u
= - \nabla p
+ \nabla^2 u
$$
Here. all variables have been normalized:
- Time normalisation: $\frac{L^2}{\nu}$
- Length normalisation: $L$
- Velocity normalisation: $\frac{\nu}{L}$
- Pressure normalisation: $\frac{\rho \nu^2}{L^2}$

We will solve this example, which is part of the openFoam tutorial, with the `icoFoam` solver.

----
## Solution strategy

Usually, a problem is broken down into the steps
- Mesh generation
- Flow calculation
- Transport calculation
- Post-processing

----
## Syntax

Equation syntax in c++ code closely follows the mathematical syntax:
    
- **Time derivative:** ${{d}\over{dt}} \Phi$

`fvc::ddt(phi)`

- **Gradient:** $\nabla \cdot \Phi$

`fvc::grad(phi)`

- **Laplacian:**

`fvc::laplacian(phi)`

- **Linearised source:** $s \Phi$

`fvc::Sp(s,phi)`

The preamble `fvc::` calls a **finite-volume calculus scheme** for the equations.

For implicit terms, a **finite-volume method** is chosen with the prefix `fvm::`.

----
## Dimensions

Often, an array for **dimensions** is present, such as: `[0 2 -2 0 0 0 0]`.

The entries describe **weights** for SI units in the form: `[kg m s K mol A cd]`


Example is for kinematic pressure $\bar{p}$: $ m^{2}s^{-2}$

----
## Directory structure

Each model run has its own directory, and the structure is fixed. Three main directories are needed
to start a run, ${\bf 0}$ for initial fields, **constant** for fixed parameter values and storage of the mesh,
and **system** for run control.

- **0**
    * U
    * p
    * T
- **constant**
    - transportProperties
    - ...
    - **polymesh**
- **system**
    - blockMeshDict
    - fvSchemes
    - fvSolution

----
### `system/blockMeshDict`

Example for a rectangular block, 1x10x0.1 m in size (x,y,z directions). In this case, the $z$ direction
is a dummy direction, because the problem is solved as 2D problem, but because `openFOAM` always calculates
in 3D, the third direction must be assinged as dummy direction.

This dictionary is located in the **system** directory.
- We are not using scaling.
- X Y Z dimensions: 1.0 10. 0.1
- We are using one single block with uniform grading.
- Cells in the X Y , and Z directions: 10 x 100 x 1 (there is only one cell in the Z direction because
the mesh is 2D).
- All edges are straight lines by default.

- The boundary patches outlet and inlet are of base type patch.
- The boundary patches left and right are of base type wall.
- The boundary patches top and bottom are of base type empty.
- Later on , we will assign the primitive type boundary conditions (numerical values), in the field files found in the directory ${\bf 0}$
- We do not need to merge faces (we have one single block).

<details><summary><b>> Show blockMeshDict</b></summary>
    
~~~
convertToMeters 1;

vertices
(
    (-0.5   0.0  -0.05)
    ( 0.5   0.0  -0.05)
    ( 0.5  10.0  -0.05)
    (-0.5  10.0  -0.05)
    (-0.5   0.0   0.05)
    ( 0.5   0.0   0.05)
    ( 0.5  10.0   0.05)
    (-0.5  10.0   0.05)
);

blocks
(
    hex (0 1 2 3 4 5 6 7) ( 10 100 1) simpleGrading (1 1 1)
);

edges
(
);

boundary
(
    inlet
    {
        type patch;
        faces
        (
            (0 1 5 4)
        );
    }
    outlet
    {
        type patch;
        faces
        (
            (2 3 7 6)
        );
    }
    left
    {
        type wall;
        faces
        (
            (0 4 7 3)
        );
    }
    right
    {
        type wall;
        faces
        (
            (1 2 6 5)
        );
    }
    bottom
    {
        type empty;
        faces
        (
            (0 3 2 1)
        );
    }
    top
    {
        type empty;
        faces
        (
            (4 5 6 7)
        );
    }
);

mergePatchPairs
(
);
~~~
</details>

----
###  `constant/transportProperties`

- This dictionary file is located in the directory constant.
- In this file we set the **kinematic viscosity** (nu, $\nu$).
The kinematic viscosity $\nu$ [m$^2$/s] is related to the **dynamic viscosity** $\eta$ [Pas] through the
**density** $\rho$ [kg/m$^3$]:
$$
\nu = {{\eta}\over{\rho}}
$$

<details><summary><b>> Show constant/transportProperties</b></summary>
    
~~~
nu nu [ 0 2 -1 0 0 0 0 ] 0.01;
~~~
</details>

----
###  `0/p`

Initial condition for pressure [m$^2$/s$^2$]. This is relative pressure, $\bar{p}={{p}\over{\rho}}$.

- We are using uniform initial conditions and the numerical value is 0 (keyword internalField). 
- For the inlet patch, we are using a zeroGradient boundary condition (we are just
extrapolating the internal values to the boundary face).
- For the outlet patch, we are using a fixedValue boundary condition with a numerical value equal to 0. 
Notice that we are using macro expansion to assign the numerical value ($internalField is equivalent to uniform 0).
- For the top patch, we are using a zeroGradient boundary condition (we are just extrapolating the internal values to the boundary face).
- For the left and right patch, we are using a zeroGradient boundary condition (we are just
extrapolating the internal values to the boundary face).
- For the top and bottom patches, we use an empty boundary condition. This boundary
condition is used for 2D simulations. These two patches are normal to the direction where we
assigned 1 cell ( Z direction).
- At this point, if you take some time and compare the
files 0/U and 0/p with the file constant/polyMesh/boundary , you will see that the name and type of each primitive type patch (the patch defined in 0 ), is consistent with the base type patch (the patch defined in the file
constant/polyMesh/boundary

<details><summary><b>> Show 0/p</b></summary>

~~~
dimensions      [0 2 -2 0 0 0 0];

internalField   uniform 0;

boundaryField
{
    inlet
    {
        type            zeroGradient;
    }

    outlet
    {
        type            fixedValue;
        value           $internalField;
    }
    left
    {
    type        zeroGradient;
    }

    right
    {
    type        zeroGradient;
    }

    bottom
    {
        type            empty;
    }

    top
    {
        type            empty;
    }
}   
~~~
</details>

----
###  `0/U`

Initial condition for velocities [m/s].

- We are using uniform initial conditions and the numerical value is ( 0 0 0 ) (keyword internalField).
- For the inlet patch, we are using a fixedValue boundary condition with a numerical value equal to ( 1 0 0 )
- For the outlet patch, we are using a zeroGradient boundary condition (we are just extrapolating the internal values to the boundary
face).
- The left and right patch is a no slip wall, therefore we impose a velocity of ( 0 0 0 ) at the wall.
- For the front and back patches, we use an empty boundary condition. This boundary
condition is used for 2D simulations. These two patches are normal to the direction where we
assigned 1 cell ( Z direction).
- At this point, if you take some time and compare the files 0/U and 0/p with the file
constant/polyMesh/boundary , you will see that the name and type of each primitive type patch (the
patch defined in 0 ), is consistent with the base type patch (the patch defined in the file
constant/polyMesh/boundary.

<details><summary><b>> Show 0/U</b></summary>
    
~~~
dimensions      [0 1 -1 0 0 0 0];

internalField   uniform (0 0 0);

boundaryField
{
    inlet
    {
        type            fixedValue;
        value           uniform (0 1 0);
    }

    outlet
    {
    type        zeroGradient;
    }

    left
    {
        type            fixedValue;
        value           uniform (0 0 0);
    }

    right
    {
        type            fixedValue;
        value           uniform (0 0 0);
    }

    bottom
    {
        type            empty;
    }
    top
    {
        type            empty;
    }
}
~~~
</details>

----
### `system/controlDict`

- This case starts from time 0 ( startTime ).
- It will run up to 20 seconds ( endTime ).
- The time step of the simulation is 0.05 seconds ( deltaT
- It will write the solution every second ( writeInterval ) of
simulation time ( runTime ).
- It will keep all the solution directories ( purgeWrite ).
- It will save the solution in ascii format ( writeFormat ).
- The write precision is 8 digits ( writePrecision ). It will only save
eight digits in the output files.
- And as the option runTimeModifiable is on, we can modify all
these entries while we are running the simulation.

<details><summary><b>> system/controlDict</b></summary>
    
~~~
application     icoFoam;

startFrom       startTime;

startTime       0;

stopAt          endTime;

endTime         20;

deltaT          0.05;

writeControl    runTime;

writeInterval   1;

purgeWrite      0;

writeFormat     ascii;

writePrecision  8;

writeCompression off;

timeFormat      general;

timePrecision   6;

runTimeModifiable true;
~~~
</details>

----
### `system/fvSchemes`

- In this case, for time discretization ( ddtSchemes ) we are using the Euler method.
- For gradient discretization ( gradSchemes ) we are using the Gauss linear method.
- For the discretization of the convective terms ( divSchemes ) we are using linear interpolation method for the term div(phi,U)
- For the discretization of the Laplacian ( laplacianSchemes and snGradSchemes ) we are using the Gauss linear method with orthogonal corrections.
- This method is second order accurate but oscillatory.
- Remember, at the end of the day we want a solution that is second order accurate.

<details><summary><b>> system/fvSchemes</b></summary>
    
~~~
ddtSchemes
{
    default         Euler;
}

gradSchemes
{
    default         Gauss linear;
    grad(p)         Gauss linear;
}

divSchemes
{
    default         none;
    div(phi,U)      Gauss linear;
}

laplacianSchemes
{
    default         Gauss linear orthogonal;
}

interpolationSchemes
{
    default         linear;
}

snGradSchemes
{
    default         orthogonal;
}
~~~
</details>

----
### `system/fvSolution`

- To solve the pressure ( p ) we are using the GAMG method with an absolute tolerance of 1e 6 and a relative tolerance relTol of 0.01 (the solver will stop iterating when it meets any of the conditions).
- The entry pFinal refers to the final correction of the PISO loop.
In this case, we are using a tighter convergence criteria in the last iteration. Notice that we are using macro expansion ($p) to copy the entries from the sub dictionary p
- To solve U we are using the linear solver PBiCG and DILU preconditioner , with an absolute tolerance of 1e 8 and a
relative tolerance relTol of 0 (the solver will stop iterating when it meets any of the conditions).
- Solving for the velocity is relative inexpensive, whereas solving for the pressure is expensive.


<details><summary><b>> system/fvSolution</b></summary>

~~~
solvers
{
    p
    {
        solver           GAMG;
        tolerance        1e-6;
        relTol           0.01;
        smoother         GaussSeidel;
        nPreSweeps       0;
        nPostSweeps      2;
        cacheAgglomeration on;
        agglomerator     faceAreaPair;
        nCellsInCoarsestLevel 100;
        mergeLevels      1;
    }

    pFinal
    {
        $p;
        relTol          0;
    }
U
    {
        solver          PBiCG;
        preconditioner  DILU;
        tolerance       1e-08;
        relTol          0;
    }
}

PISO
{
    nCorrectors     1;
    nNonOrthogonalCorrectors 1;
    //pRefCell        0;
    //pRefValue       0;
}
~~~
<details>

... done